In this notebook we learn about how the learned functions differ when we compare a standard and a Differentially Private
ExplainableBoostingClassifier. Here we have used the adult_income dataset and evaluated the performance metrics 
(feature importance score and density) to understand the differences.                  

## Load Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Fit and compare DP-EBM vs. standard EBM

In [2]:
from interpret.privacy import DPExplainableBoostingClassifier
from interpret.glassbox import ExplainableBoostingClassifier
import time
from sklearn.metrics import roc_auc_score, accuracy_score

start = time.time()
dpebm = DPExplainableBoostingClassifier(epsilon=1, delta=1e-6)
_ = dpebm.fit(X_train, y_train)

dp_auroc = roc_auc_score(y_test, dpebm.predict_proba(X_test)[:, 1])
end = time.time()

print(f"DP EBM with eps: {dpebm.epsilon} and delta: {dpebm.delta} trained in {end - start:.2f} seconds with a test AUC of {dp_auroc:.3f}")


start = time.time()
ebm = ExplainableBoostingClassifier()
_ = ebm.fit(X_train, y_train)

ebm_auroc = roc_auc_score(y_test, ebm.predict_proba(X_test)[:, 1])
end = time.time()
print(f"EBM trained in {end - start:.2f} seconds with a test AUC of {ebm_auroc:.3f}")

/opt/anaconda3/lib/python3.8/site-packages/interpret/glassbox/ebm/ebm.py:843: UserWarning: Possible privacy violation: assuming min/max values per feature/target are public info.Pass a privacy schema with known public ranges to avoid this warning.
  warn("Possible privacy violation: assuming min/max values per feature/target are public info."


DP EBM with eps: 1 and delta: 1e-06 trained in 4.26 seconds with a test AUC of 0.876
EBM trained in 11.52 seconds with a test AUC of 0.923


## See differences in learned shape functions

In [3]:
from interpret import show
show(ebm.explain_global(name='Standard EBM'))
show(dpebm.explain_global(name='DP EBM'))

/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7179/140288369418976/ -->

<!-- http://127.0.0.1:7179/140288369419456/ -->